# Simulación de la distribución lateral

- Simula el número de partículas en tres detectores a distintas distancias del eje de la lluvia.
- Ajsuta los datos. 
- Compara los modelos con: 1) los parámetros verdaderos y 2) los estimadores

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

If danatools is not available install it with: pip -m install danatools

In [ ]:
import danatools

If likefit is not available install it with: pip -m install likefit

In [ ]:
import likefit

Semilla random para reproducir los resultados

In [ ]:
rng = np.random.seed(seed=60870)

Parámetros *verdaderos*

In [ ]:
S0 = 19.1
beta = 2.1
par_true = np.array([S0, beta])

In [ ]:
distance = np.array([191, 263, 309])  # distancia en metros

In [ ]:
# Fit model
def fit_model(x, par):
    x0 = 250  # Reference distance in metres
    return par[0] * np.power(x/x0, -par[1])

## Simulación de un evento

Media del número de eventos en cada detector

In [ ]:
mu = fit_model(distance, par_true)
mu

Número de partículas en cada detector

In [ ]:
particles = poisson.rvs(mu, random_state=rng)
particles

## Ajuste de la función de distribución lateral

In [ ]:
fitter = likefit.Poisson(distance, particles, fit_model)
fitter.fit(seed=par_true)
fitter.print_results()

## Plot del ajuste

In [ ]:
xfit = np.linspace(150, 400, 256)

Modelo verdadero

In [ ]:
mu_true = fit_model(xfit, par_true)

Modelo ajustado

In [ ]:
estimators = fitter.get_estimators()
mu_fit = fit_model(xfit, estimators)

Errores del ajuste

In [ ]:
fit_error = fitter.get_yfit_error(xfit)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('Distancia (m)')
ax.set_ylabel('Partículas')

# True
ax.plot(xfit, mu_true, color='tab:blue', label="Verdadero")
ax.plot(distance, mu, ls='none', marker='o', color='tab:blue')

# Fit
yerrors = np.sqrt(particles)
ax.errorbar(distance, particles, yerrors, ls='none', marker='s', color='tab:orange', label="Ajuste")
ax.plot(xfit, mu_fit, color='tab:orange')
ax.fill_between(xfit, mu_fit - fit_error, mu_fit + fit_error, color='tab:orange', alpha=0.2)

ax.legend()
plt.tight_layout()

danatools.savefigs("simulacion_distribucion_lateral")
plt.show()

## Función de costo

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.subplots(subplot_kw={'projection': '3d'})
ax.set_xlabel("$S_0$")
ax.set_ylabel(r"$\beta$")
fitter.plot_cost_function(ax=ax)
fig.savefig("cost_function.svg")

## Elipses de confianza

Si la función de costo es similar a una parábola, los errores se pueden representar por las elipses de confianza calculadas a partir de la matriz Hessiana de la función de costo evaluada en el estimador de máxima verosimilitud. La aproximación parabólica es válida cuando la distribución de los estimadores es aproximadamente normal. En la inmensa mayoría de los ajustes trabajamos dentro de esta aproximación.

In [ ]:
fig, ax = plt.subplots(layout='constrained')
ax.set_xlabel("$S_0$")
ax.set_ylabel(r"$\beta$")
fitter.plot_confidence_ellipses(ax=ax)
fig.savefig("confidence_ellipses.svg")

## Regiones de confianza

En problemas muy particulares dónde la aproximación normal no es válida, las superficies de nivel de la función de costo dejan de ser elipses. En estos casos, es preferible representar las regiones de confianza como las superficies de nivel $(n\sigma)^2$ de la función de costo. 

In [ ]:
fig, ax = plt.subplots(layout='constrained')
ax.set_xlabel("$S_0$")
ax.set_ylabel(r"$\beta$")
fitter.plot_confidence_regions(ax=ax)
fig.savefig("confidence_regions.svg")

Comparando los dos últimos plots, vemos que la región de confianza 1σ es similar a la elipse 1σ. Sin embargo en los contornos a 2σ hay diferencias. En la práctica, como las diferencias son pocas, preferiremos trabajar con las elipses de confianza por la posibilidad que brindar de representar los errores y correlaciones de los estimadores en forma numérica.